In [264]:
import pandas as pd
import re
import requests
import difflib
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [334]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get('https://fantasyfootballcalculator.com/adp', headers=header)
ffc = pd.read_html(r.text, header = 0); ffc = ffc[0]

ffc = ffc[['Pick', 'Name', 'Overall', 'Std.Dev', 'High', 'Low']]
ffc = ffc.rename(columns = {'Name': 'name'})



fp = pd.read_html('https://www.fantasypros.com/nfl/adp/overall')[0]
fp['Player Team (Bye)'] = fp.iloc[:,1].astype('str')
name = []
for i in range(0,len(fp)):
    try: 
        name.append (re.findall('(.*?)\s[A-Z][A-Z]', fp.iloc[i,1])[0])
    except:
        name.append("")
    

fp['name'] = name
fp = fp.drop('Player Team (Bye)', axis = 1)

stopwords = [" Jr.", " II", " III", " IV", "Defense"]



fp['name'].apply(lambda x: [item for item in x if item not in stopwords])

pat = r'\b(?:{})\b'.format('|'.join(stopwords))
fp['name'] = fp['name'].str.replace(pat, '')
ffc['name'] = ffc['name'].str.replace(pat, '')
fp['name'] = fp['name'].str.replace('.', '')
ffc['name'] = ffc['name'].str.replace('.', '')


fp['name'] = fp['name'].str.upper()
ffc['name'] = ffc['name'].str.upper()

rem_name = []
remainder = ffc[-ffc['name'].isin(fp['name'])]
for i in range(0, len(remainder)):
    rem_name.append( difflib.get_close_matches(remainder.iloc[i,1], fp['name'], cutoff = .55)[0])

indexer = ffc[-ffc['name'].isin(fp['name'])].index
ffc.loc[indexer, "name"] = rem_name



df = ffc.merge(fp)
proj = pd.read_csv('../projections.csv')
rem_name = []
for i in range(0, len(df)):
    rem_name.append( difflib.get_close_matches(df.iloc[i,1], proj['name'], cutoff = .4)[0])

df["name"] = rem_name
df = df[['name', 'Overall', 'AVG', 'Std.Dev', 'High', 'Low']]


df['High'] = df['High'] + (12 * (df['High'] - 1.01) ) -.01
df['Low'] = df['Low'] + (12 * (df['Low'] - 1.01) )

df.to_csv('../adp.csv')


In [335]:
proj['name']

0                       BUFFALO
1                  INDIANAPOLIS
2                         MIAMI
3                   NEW ENGLAND
4                      NEW YORK
5                    CINCINNATI
6                     CLEVELAND
7                     TENNESSEE
8                  JACKSONVILLE
9                    PITTSBURGH
10                       DENVER
11                  KANSAS CITY
12                      OAKLAND
13                  LOS ANGELES
14                      SEATTLE
15                       DALLAS
16                     NEW YORK
17                 PHILADELPHIA
18                      ARIZONA
19                   WASHINGTON
20                      CHICAGO
21                      DETROIT
22                    GREEN BAY
23                    MINNESOTA
24                    TAMPA BAY
25                      ATLANTA
26                     CAROLINA
27                  LOS ANGELES
28                  NEW ORLEANS
29                SAN FRANCISCO
30                    BALTIMORE
31      

In [328]:
proj

,Unnamed: 0,last_name,first_name,pos,id,points,rank,pos_rank,sd_pts,avg_type,name
0,1,Bills,Buffalo,DST,501,107.638000,114.0,12.0,14.181069,robust,BUFFALO
1,2,Colts,Indianapolis,DST,502,109.622000,106.0,6.0,9.540531,robust,INDIANAPOLIS
2,3,Dolphins,Miami,DST,503,93.159500,160.0,31.0,8.406342,robust,MIAMI
3,4,Patriots,New England,DST,504,108.018500,112.0,11.0,2.520420,robust,NEW ENGLAND
4,5,Jets,New York,DST,505,100.796000,136.0,23.0,7.116480,robust,NEW YORK
5,6,Bengals,Cincinnati,DST,506,95.110000,158.0,30.0,5.693184,robust,CINCINNATI
6,7,Browns,Cleveland,DST,507,109.202500,108.0,8.0,5.752488,robust,CLEVELAND
7,8,Titans,Tennessee,DST,508,100.246500,142.0,24.0,8.006040,robust,TENNESSEE
8,9,Jaguars,Jacksonville,DST,509,106.019000,123.0,18.0,10.653964,robust,JACKSONVILLE
9,10,Steelers,Pittsburgh,DST,510,110.351000,103.0,4.0,1.875489,robust,PITTSBURGH


In [329]:
df

,name,Overall,AVG,Std.Dev,High,Low
0,SAQUON BARKLEY,1.5,1.0,0.7,1.00,1.40
1,ALVIN KAMARA,2.4,2.3,0.8,1.00,1.53
2,CHRISTIAN MCCAFFREY,2.5,2.7,0.9,1.00,1.53
3,EZEKIEL ELLIOTT,4.5,4.0,1.4,1.00,2.31
4,DAVID JOHNSON,5.2,6.0,1.3,1.26,2.31
5,DEANDRE HOPKINS,6.0,5.7,1.8,1.00,2.44
6,LE'VEON BELL,7.3,6.7,2.3,1.00,14.53
7,DAVANTE ADAMS,7.9,8.7,2.0,1.00,14.14
8,NICK CHUBB,8.5,14.0,2.3,1.13,14.53
9,JAMES CONNER,9.8,9.3,2.2,1.39,14.53
